In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import sys
import json
import time
from shapely.geometry import box,LineString,Point,MultiPoint,MultiLineString,MultiPolygon,Polygon
from shapely.ops import unary_union
import socket
from collections import defaultdict
from numba import prange
import gc
current_dir = os.path.join(os.getcwd()) 
mother_path = os.path.abspath(os.path.join(current_dir, os.pardir))
print('mother_path:', mother_path)
sys.path.append(os.path.join(mother_path, 'PreProcessing'))
sys.path.append(os.path.join(mother_path))
from FittingProcedures import *
from plot import *
from Potential import *
from Grid import *
from PreprocessingObj import *
from ODfromfma import *
import time
from GeometrySphere import *
from Polycentrism import *
from PolycentrismPlot import *
from ModifyPotential import *
from MainPolycentrism import *

#import rustworkx as rw
import ast
## BASIC PARAMS
gc.set_threshold(10000,50,50)
plt.rcParams.update({
    "text.usetex": False,
})

if socket.gethostname()=='artemis.ist.berkeley.edu':
    TRAFFIC_DIR = '/home/alberto/LPSim/traffic_phase_transition'
else:
    TRAFFIC_DIR = os.getenv('TRAFFIC_DIR')
name = 'BOS'
grid_size = 0.02
hexagon_resolution = 8
dir_grid = os.path.join(TRAFFIC_DIR,'data','carto',name,'grid',str(grid_size))

## BASIC NEEDED OBJECTS
SFO_obj = GeometricalSettingsSpatialPartition(name,TRAFFIC_DIR)
bbox = SFO_obj.bounding_box
minx, miny, maxx, maxy = bbox
dir_geometries = os.path.join(TRAFFIC_DIR,'data','carto','{0}'.format(name))
grid = GetGrid(grid_size,SFO_obj.bounding_box,'EPSG:4326',dir_geometries)
grid = GetBoundariesInterior(grid,SFO_obj)
df_distance,FoundDistMat = GetDirectionMatrix(dir_geometries,grid_size)
Tij = GetODGrid(dir_geometries,str(grid_size))
lattice = nx.read_graphml(os.path.join(dir_grid,"centroid_lattice.graphml"))


mother_path: /home/alberto/LPSim/traffic_phase_transition/scripts
Initialize Grid: 0.02
ALREADY COMPUTED


In [11]:
def EstimateBestInitialGuess2Params(bins,n,label):
    alpha = -1.
    binsMax = bins[-1]
    binsMin = bins[1]
    Integral = np.sum([n[i]*bins[i+1] for i in range(len(bins[1:]))])
    if label == 'powerlaw':
        A = Integral*alpha*(binsMax**(alpha+1)-binsMin**(alpha+1))**(-1)
        return A,alpha
    elif label == 'exponential':
        A = Integral*(np.exp(-alpha*binsMin)-np.exp(-alpha*binsMax))**(-1)
        return A,alpha
    else:
        return None
def Pt0GivenDLC0(edges):
    t0s = edges.apply(lambda x: x['length']/x['speed_mph'],axis=1)
    nTime,binsTime = np.histogram(t0s,bins=100)
    A,alpha = EstimateBestInitialGuess2Params(binsTime, nTime, 'powerlaw')
    fitPl = Fitting(binsTime,nTime,label = 'powerlaw',initial_guess = (A,alpha),maxfev = 10000)
    A,alpha = EstimateBestInitialGuess2Params(binsTime, nTime, 'expoenntial')
    fitExp = Fitting(binsTime,nTime,label = 'exponential',initial_guess = (A,alpha),maxfev = 10000)
    
def PlotProbability(binsTime,nTime,fitPl,fitExp):
    '''
        Plot the probability of the time of travel
    '''
    fig,ax = plt.subplots(1,1,figsize=(8,8))
    ax.plot(binsTime[1:],nTime/np.sum(nTime),label='Empirical')
    ax.plot(binsTime[1:],fitPl[0](binsTime[1:]),label='Power Law')
    ax.plot(binsTime[1:],fitExp[0](binsTime[1:]),label='Exponential')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.legend()
    plt.show()
    

{'crs': 'epsg:4326', 'city': 'BOS', 'config_dir_local': '/home/alberto/LPSim/traffic_phase_transition/config', 'tiff_file_dir_local': '/home/alberto/LPSim/traffic_phase_transition/data/carto/tiff_files', 'shape_file_dir_local': '/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/shape_files', 'ODfma_dir': '/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/ODfma', 'save_dir_local': '/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS', 'save_dir_server': '/home/alberto/LPSim/LivingCity/berkeley_2018/BOS', 'GraphFromPhml': <networkx.classes.multidigraph.MultiDiGraph object at 0x7f5a9b609c40>, 'gdf_polygons':      tractid    my_area                                           geometry
0     630400   9.988907  POLYGON ((-71.35236 41.93142, -71.35281 41.931...
1     631100   8.289611  POLYGON ((-71.35281 41.93117, -71.35236 41.931...
2     631300   8.960974  POLYGON ((-71.29017 41.94444, -71.29021 41.944...
3     631200  17.019618  POLYGON ((-71.30786 41.96025, -71.

In [ ]:
def MorletWavelet(x, sigma):
    # Compute the complex exponential
    complex_exponential = np.exp(2j * np.pi * x / sigma)
    # Take the real part to get the cosine
    cosine = complex_exponential.real
    
    # Compute the Gaussian and multiply by the cosine
    return np.exp(-x**2 / 2) * cosine